# IBM AI OpenScale & Microsoft Azure Machine Learning

<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/image.png" align="left" alt="banner" width="100">


This notebook shows how to log the payload for the model deployed on Azure using AI OpenScale python client.

Contents
- [1. Setup](#setup)
- [2. Binding machine learning engine](#binding)
- [3. Subscriptions](#subscription)
- [4. Scoring and payload logging](#scoring)
- [5. Data Mart](#datamart)

<a id="setup"></a>
## 1. Setup

### 1.1 Installation and authentication

In [ ]:
!pip install --upgrade ibm-ai-openscale

Import and initiate.

In [2]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.engines import *

### ACTION: Get `data_mart_id` (GUID) and `apikey`

How to get api key using bluemix console:
```
bx login --sso
bx iam api-key-create 'my_key'
```

How to get DATA_MART_ID (this is AI OpenScale instance GUID)
```
bx resource service-instance ‘instance_name’
```


In [ ]:
aios_credentials = {
  "data_mart_id": "***",
  "url": "https://api.aiopenscale.cloud.ibm.com",
  "apikey": "***"
}

In [14]:
# The code was removed by Watson Studio for sharing.

In [15]:
client = APIClient(aios_credentials)

In [16]:
client.version

'1.0.1'

In [ ]:
postgres_credentials = {
    "db_type": "postgresql",
    "uri_cli_1": "xxx",
    "maps": [],
    "instance_administration_api": {
        "instance_id": "xxx",
        "root": "xxx",
        "deployment_id": "xxx"
    },
    "name": "xxx",
    "uri_cli": "xxx",
    "uri_direct_1": "xxx",
    "ca_certificate_base64": "xxx",
    "deployment_id": "xxx",
    "uri": "xxx"
}

In [19]:
# The code was removed by Watson Studio for sharing.

Create schema for data mart.

In [18]:
schemaName = 'azure_model_data'

In [28]:
import psycopg2


hostname = postgres_credentials['uri'].split('@')[1].split(':')[0]
port = postgres_credentials['uri'].split('@')[1].split(':')[1].split('/')[0]
user = postgres_credentials['uri'].split('@')[0].split('//')[1].split(':')[0]
password = postgres_credentials['uri'].split('@')[0].split('//')[1].split(':')[1]
dbname = 'compose'

conn_string = "host=" + hostname + " port=" + port + " dbname=" + dbname + " user=" + user + " password=" + password
conn = psycopg2.connect(conn_string)
conn.autocommit = True
cursor = conn.cursor()
try:
    query = "drop schema " + schemaName + " cascade"
    cursor.execute(query)
except:
    pass
finally:    
    try:
        query = "create schema " + schemaName
        cursor.execute(query)
    finally:    
        conn.close()

### 1.2 DataMart setup

In [29]:
client.data_mart.setup(postgres_credentials=postgres_credentials, schema=schemaName)

In [30]:
data_mart_details = client.data_mart.get_details()

<a id="binding"></a>
## 2. Bind machine learning engines

### 2.1 Bind Microsoft Azure as `GENERIC` engine
**NOTE:** GENERIC means that this is just metada (abstraction) - there is no direct integration with that service

In [32]:
binding_uid = client.data_mart.bindings.add('My Azure', GenericMachineLearningInstance())

In [33]:
bindings_details = client.data_mart.bindings.get_details()

In [34]:
client.data_mart.bindings.list()

generic_instance_id_a9dac2de-2c5d-4898-9527-c58d6eee89d5,My Azure,generic_machine_learning,2018-09-19T12:37:16.796Z


<a id="subsciption"></a>
## 3. Subscriptions

### 3.1 Add `GENERIC` subscriptions

In [35]:
subscription = client.data_mart.subscriptions.add(GenericAsset(name='Azure deployment of product line sample model', binding_uid=binding_uid))

Creating default deployment for generic asset with uid: generic_deployment_uid_00a19490-7114-4f01-8f70-5c03e1be4d18


#### Get subscriptions list

In [36]:
subscriptions = client.data_mart.subscriptions.get_details()

In [37]:
subscriptions_uids = client.data_mart.subscriptions.get_uids()
print(subscriptions_uids)

['generic_uid_17fa0ff5-abe9-47e4-ab81-b9a9007669e7']


#### List subscriptions

In [38]:
client.data_mart.subscriptions.list()

generic_uid_17fa0ff5-abe9-47e4-ab81-b9a9007669e7,Azure deployment of product line sample model,model,generic_instance_id_a9dac2de-2c5d-4898-9527-c58d6eee89d5,generic_uid_17fa0ff5-abe9-47e4-ab81-b9a9007669e7,2018-09-19T12:37:40Z


### 3.2 Configure subscription

#### Enable payload logging in subscription

In [39]:
subscription.payload_logging.enable()

#### Get details of enabled payload logging

In [40]:
subscription.payload_logging.get_details()

{'enabled': True,
 'parameters': {'dynamic_schema_update': True,
  'table_name': 'azure_model_data.Payload_generic_uid_17fa0ff5-abe9-47e4-ab81-b9a9007669e7'}}

<a id="scoring"></a>
# 4. Scoring and payload logging

## 4.1 Score the model (Microsoft Azure)

In [41]:
import urllib.request
import json

data = {
        "Inputs": {
                "input1":
                [
                    {
                            'GENDER': "F",   
                            'AGE': "27",   
                            'MARITAL_STATUS': "Single",   
                            'PROFESSION': "Professional",   
                            'PRODUCT_LINE': "Personal Accessories",   
                    }
                ],
        },
    "GlobalParameters":  {
    }
}

body = str.encode(json.dumps(data))

url = 'https://ussouthcentral.services.azureml.net/workspaces/1e5142d3a8ba4b51ac24bee7c65914a1/services/6bb8104a38b8492d9b71cb276c534bfd/execute?api-version=2.0&format=swagger'
api_key = 'gxGozX4Sw/Vx/DRbnthKzYTzD8QbgJX3TVTVTe54a9wAmudwqmOGXzi0SInzriUJHgvF2csDh7tsI7dLBeEuIQ==' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)
response = urllib.request.urlopen(req)

result = response.read()
result = json.loads(result.decode())['Results']['output1'][0]
print(json.dumps(result, indent=2))

{
  "Scored Probabilities for Class \"Mountaineering Equipment\"": "0.0573424553534177",
  "Scored Labels": "Personal Accessories",
  "PROFESSION": "Professional",
  "PRODUCT_LINE": "Personal Accessories",
  "MARITAL_STATUS": "Single",
  "GENDER": "F",
  "Scored Probabilities for Class \"Golf Equipment\"": "0",
  "Scored Probabilities for Class \"Outdoor Protection\"": "0",
  "Scored Probabilities for Class \"Camping Equipment\"": "0",
  "AGE": "27",
  "Scored Probabilities for Class \"Personal Accessories\"": "0.942657544646582"
}


## 4.2 Convert the request and response to the form supported by OpenScale.

In [42]:
input_data = data['Inputs']['input1'][0]

request = {
    'fields': list(input_data.keys()),
    'values': [[input_data[i] for i in list(input_data.keys())]]
    
}

response = {
    'fields': list(result.keys()),
    'values' : [[result[i] for i in list(result.keys())]]
}

In [43]:
print(str(request))

{'values': [['F', 'Single', 'Professional', 'Personal Accessories', '27']], 'fields': ['GENDER', 'MARITAL_STATUS', 'PROFESSION', 'PRODUCT_LINE', 'AGE']}


## 4.3 Store the request and response in payload logging table

### Python client

In [44]:
subscription.payload_logging.store(request=request, response=response)

### REST API

Get the token first.

In [64]:
token_endpoint = "https://iam.bluemix.net/identity/token"
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Accept": "application/json"
}

data = {
    "grant_type":"urn:ibm:params:oauth:grant-type:apikey",
    "apikey":aios_credentials["apikey"]
}

req = requests.post(token_endpoint, data=data, headers=headers)
token = req.json()['access_token']

In [67]:
import requests, uuid

PAYLOAD_STORING_HREF_PATTERN = '{}/v1/data_marts/{}/scoring_payloads'
endpoint = PAYLOAD_STORING_HREF_PATTERN.format(aios_credentials['url'], aios_credentials['data_mart_id'])

payload = [{
    'binding_id': binding_uid, 
    'deployment_id': subscription.get_details()['entity']['deployments'][0]['deployment_id'], 
    'subscription_id': subscription.uid, 
    'scoring_id': str(uuid.uuid4()), 
    'response': response,
    'request': request
}]


headers = {"Authorization": "Bearer " + token}
      
req_response = requests.post(endpoint, json=payload, headers = headers)

print("Request OK: " + str(req_response.ok))

Request OK: True


<a id="datamart"></a>
# 5. Get the logged data

#### Print schema of payload_logging table

In [68]:
subscription.payload_logging.print_table_schema()

scoring_id,text,-,-,-,N
scoring_timestamp,timestamp,8,-,-,N
deployment_id,text,-,-,-,N
asset_revision,text,-,-,-,N
GENDER,text,-,-,-,N
MARITAL_STATUS,text,-,-,-,N
PROFESSION,text,-,-,-,N
PRODUCT_LINE,text,-,-,-,N
AGE,text,-,-,-,N
"Scored Probabilities for Class ""Mountaineering Equipment""",text,-,-,-,N
Scored Labels,text,-,-,-,N


#### Show (preview) the table

In [69]:
subscription.payload_logging.show_table()

82c72435-590d-4d2e-baa1-78a08f16300f-1,2018-09-19 12:39:54.732069,generic_deployment_uid_00a19490-7114-4f01-8f70-5c03e1be4d18,123456,F,Single,Professional,Personal Accessories,27,0.0573424553534177,Personal Accessories,0,0,0,0.942657544646582
75590698-aeec-43cb-b18b-605447f319c8-1,2018-09-19 13:01:14.803000,generic_deployment_uid_00a19490-7114-4f01-8f70-5c03e1be4d18,None,F,Single,Professional,Personal Accessories,27,0.0573424553534177,Personal Accessories,0,0,0,0.942657544646582
778b2531-76f3-4aef-b5d8-3f037fa2a559-1,2018-09-19 13:01:23.849000,generic_deployment_uid_00a19490-7114-4f01-8f70-5c03e1be4d18,None,F,Single,Professional,Personal Accessories,27,0.0573424553534177,Personal Accessories,0,0,0,0.942657544646582
ec513f1d-d4a2-4af6-aa06-b1c08e430d97-1,2018-09-19 13:01:32.650000,generic_deployment_uid_00a19490-7114-4f01-8f70-5c03e1be4d18,None,F,Single,Professional,Personal Accessories,27,0.0573424553534177,Personal Accessories,0,0,0,0.942657544646582


#### Return the table content as pandas dataframe

In [70]:
pandas_df = subscription.payload_logging.get_table_content(format='pandas')
pandas_df

,scoring_id,scoring_timestamp,deployment_id,asset_revision,GENDER,MARITAL_STATUS,PROFESSION,PRODUCT_LINE,AGE,"Scored Probabilities for Class ""Mountaineering Equipment""",Scored Labels,"Scored Probabilities for Class ""Golf Equipment""","Scored Probabilities for Class ""Outdoor Protection""","Scored Probabilities for Class ""Camping Equipment""","Scored Probabilities for Class ""Personal Accessories"""
0,82c72435-590d-4d2e-baa1-78a08f16300f-1,2018-09-19 12:39:54.732069,generic_deployment_uid_00a19490-7114-4f01-8f70...,123456,F,Single,Professional,Personal Accessories,27,0.0573424553534177,Personal Accessories,0,0,0,0.942657544646582
1,75590698-aeec-43cb-b18b-605447f319c8-1,2018-09-19 13:01:14.803000,generic_deployment_uid_00a19490-7114-4f01-8f70...,None,F,Single,Professional,Personal Accessories,27,0.0573424553534177,Personal Accessories,0,0,0,0.942657544646582
2,778b2531-76f3-4aef-b5d8-3f037fa2a559-1,2018-09-19 13:01:23.849000,generic_deployment_uid_00a19490-7114-4f01-8f70...,None,F,Single,Professional,Personal Accessories,27,0.0573424553534177,Personal Accessories,0,0,0,0.942657544646582
3,ec513f1d-d4a2-4af6-aa06-b1c08e430d97-1,2018-09-19 13:01:32.650000,generic_deployment_uid_00a19490-7114-4f01-8f70...,None,F,Single,Professional,Personal Accessories,27,0.0573424553534177,Personal Accessories,0,0,0,0.942657544646582


---

### Authors
Lukasz Cmielowski, PhD, is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increases clients' ability to turn data into actionable knowledge.